# Import

In [27]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Web Mining Tut/Project/"
%cd /content/drive/My\ Drive/Web Mining Tut/Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data			      graph_sampled.png
Data_analysis.ipynb	      graph_test.ipynb
df_melted_fullymined_pro.csv  Preprocessing_avg_players.ipynb
dict_clear.txt		      Preprocessing_pro_players.ipynb
dict.txt		      RiotMining.ipynb
/content/drive/My Drive/Web Mining Tut/Project


In [28]:
import numpy as np
import pandas as pd
import re
import networkx as nx
from matplotlib import pyplot as plt

# Data

## Matches

In [29]:
# matches
matches = pd.read_csv('./data/match_playerPUUIDs_timestampAvgPlayers.csv', header= None)
matches.columns = ['match_id','player1', 'player2', 'player3', 'player4', 'player5', 'player6', 'player7', 'player8', 'player9', 'player10', 'timestamp']

In [30]:
# get rid of brackets 
for col in matches:
    matches[col] = matches[col].astype(str).str.replace("[\]\{\}\'\(\)\ \[]","")
#matches.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# replace nan strings with nan values
matches.replace('nan', np.nan, inplace=True)

In [32]:
# check for na values
print(matches.isna().sum())
#len(matches)

match_id      0
player1       0
player2       0
player3      21
player4      41
player5      41
player6      51
player7      51
player8      55
player9      55
player10     56
timestamp    56
dtype: int64


In [33]:
# timestamps that are null
correct = matches[matches['timestamp'].isnull()]
mask = correct[correct['timestamp'].isnull()].player10.isnull()
correct.loc[mask,['timestamp','player9']] = correct.loc[mask,['player9','timestamp']].values

correct2 = correct[correct['timestamp'].isnull()]
correct = correct[correct['timestamp'].notnull()]

correct2[correct2['timestamp'].isnull()].player8.isnull()
mask = correct2[correct2['timestamp'].isnull()].player8.isnull()
correct2.loc[mask,['timestamp','player7']] = correct2.loc[mask,['player7','timestamp']].values

correct3 = correct2[correct2['timestamp'].isnull()]
correct2 = correct2[correct2['timestamp'].notnull()]

correct3[correct3['timestamp'].isnull()].player7.isnull()
mask = correct3[correct3['timestamp'].isnull()].player7.isnull()
correct3.loc[mask,['timestamp','player6']] = correct3.loc[mask,['player6','timestamp']].values

correct4 = correct3[correct3['timestamp'].isnull()]
correct3 = correct3[correct3['timestamp'].notnull()]

correct4[correct4['timestamp'].isnull()].player6.isnull()
mask = correct4[correct4['timestamp'].isnull()].player6.isnull()
correct4.loc[mask,['timestamp','player5']] = correct4.loc[mask,['player5','timestamp']].values

correct5 = correct4[correct4['timestamp'].isnull()]
correct4 = correct4[correct4['timestamp'].notnull()]

correct5[correct5['timestamp'].isnull()].player5.isnull()
mask = correct5[correct5['timestamp'].isnull()].player5.isnull()
correct5.loc[mask,['timestamp','player4']] = correct5.loc[mask,['player4','timestamp']].values

correct6 = correct5[correct5['timestamp'].isnull()]
correct5 = correct5[correct5['timestamp'].notnull()]

correct6[correct6['timestamp'].isnull()].player4.isnull()
mask = correct6[correct6['timestamp'].isnull()].player4.isnull()
correct6.loc[mask,['timestamp','player3']] = correct6.loc[mask,['player3','timestamp']].values

correct7 = correct6[correct6['timestamp'].isnull()]
correct6 = correct6[correct6['timestamp'].notnull()]

correct7[correct7['timestamp'].isnull()].player3.isnull()
mask = correct7[correct7['timestamp'].isnull()].player3.isnull()
correct7.loc[mask,['timestamp','player2']] = correct7.loc[mask,['player2','timestamp']].values
correct7 = correct7[correct7['timestamp'].notnull()]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [34]:
# check if all rows catched
#len(correct) + len(correct2) + len(correct3) + len(correct4) + len(correct5) + len(correct6) + len(correct7)

In [35]:
corrects = pd.concat([ correct, correct2, correct3, correct4, correct5, correct6, correct7])
corrects["timestamp"]= pd.to_datetime(corrects["timestamp"], format="%Y-%m-%dT%H:%M:%S")

In [36]:
# combine to one dataframe
matches["timestamp"]= pd.to_datetime(matches["timestamp"], format="%Y-%m-%dT%H:%M:%S")
new_matches = matches[matches['timestamp'].notna()]
new_matches = pd.concat([new_matches, correct, correct2, correct3, correct4, correct5, correct6, correct7])

In [37]:
# check 
#new_matches.head()
#matches[matches['match_id']=='5777687631']
#new_matches[new_matches['match_id']=='5838219052']

##Melted

In [38]:
# melt matches
matches_melted = (new_matches.melt(['match_id', 'timestamp'], 
                       var_name='player_number',
                       value_name='player_puuid'))
#len(matches_melted)

In [39]:
#remove empty cells (empty puuids, empty matches in general)
matches_melted['player_puuid'].replace(' ', np.nan, inplace=True)
matches_melted['player_puuid'].isna().sum()
matches_melted = matches_melted[matches_melted['player_puuid'].notna()]

In [40]:
#get rid of BOTs
#bots = matches_melted[matches_melted['player_puuid'].str.len()<5]
#len(bots)
matches_melted = matches_melted[matches_melted['player_puuid'].str.len()>5]
matches_melted.head()

,match_id,timestamp,player_number,player_puuid
0,5857231455,2022-05-05 13:18:51+00:00,player1,ATaIws30cj0UfWN179fxT95zfYUrAAWNPhJHRh6h8_IIYR...
1,5857099343,2022-05-05 12:47:24+00:00,player1,qxzKB-XDos1g8BVytNYaxzC5QvaXnJ2FHPkMtv0rWG_wmv...
2,5856620450,2022-05-04 23:45:01+00:00,player1,YEFUr568ruap9KJSRAeT29Fkb34anUma7sX9OeYRxs0-Pi...
3,5856716446,2022-05-04 23:09:33+00:00,player1,NFuiAvF0C9YMCBBfgJpDYAaaZqS8IFhnKX5_02QBeZIREl...
4,5855943984,2022-05-04 18:04:03+00:00,player1,J87wGe7JHWF07jC521xbEiFuRnHFqC5NPbwM_IUEjOgfzj...


In [41]:
matches_melted.describe()

,match_id,timestamp,player_number,player_puuid
count,376015,376015,376015,376015
unique,33870,32657,10,217702
top,5857231455,2022-05-06 17:20:28+00:00,player1,jue-VFnXs4CIrR0kAfkU3dd8voN7im5Z7nJprRX-ELuJJH...
freq,20,50,37662,93


In [42]:
# check for duplicates
matches_melted[matches_melted.duplicated(subset=['match_id','timestamp'])].value_counts()

match_id    timestamp                  player_number  player_puuid                                                                  
5860226427  2022-05-07 13:56:17+00:00  player4        8VQLq6pnmGRKzzUuHDPCC1IOyq3y7oobY1qa2L6G3MXukJsAVFkK6DfKipwuReqtzWjri85WDhC_8w    2
5848185381  2022-04-29 22:42:18+00:00  player3        Ss3ixu8NGFLA6nbA4nd7xYdIxuscnpvCwf2tdUgMi87conqVP-HdB6CDaO_iv684S3NJD3IwrUeNeQ    2
5861648114  2022-05-08 13:17:37+00:00  player3        P5qpEHIQ151X9TRluAQ9JNLzR-r0F3EEw52y0n0EX-biCFIUi6w3otwQ_72QhGrLPwknbHlMssT-_A    2
                                       player2        9OF6KLutzMIfqb7m27kefiw3Yi48PtpfTtstjQ8PmWbN7XZvFHlvKOkArrYVZuL1IhV70HnB4gc8Bg    2
                                       player10       gMo9xm7Sg98pmh3tYGaJSIUBfAF3qp1JZpJEtOZJaf_0H3-EeiSMZmG4Bxr7a5gKE0KQOdwSrwhulg    2
                                                                                                                                       ..
5857035214  2022-05-05 10:11:47+00:00  

In [43]:
# get rid of duplicates
avg_matches = matches_melted.drop_duplicates(subset=["match_id", "timestamp", "player_number"])
avg_matches.describe()

,match_id,timestamp,player_number,player_puuid
count,338101,338101,338101,338101
unique,33870,32657,10,217702
top,5857231455,2022-05-07 11:44:39+00:00,player1,jue-VFnXs4CIrR0kAfkU3dd8voN7im5Z7nJprRX-ELuJJH...
freq,10,40,33870,58


## Ranks

In [44]:
# ranks
ranks = pd.read_csv('./data/fullyMinedSumonerPUUID_RankAvgPlayers.csv', header= None)
ranks.columns = ['player_puuid','rank']

In [45]:
# check for na and null values
print(ranks.isna().sum())
print(ranks.isnull().sum())

player_puuid    0
rank            0
dtype: int64
player_puuid    0
rank            0
dtype: int64


In [46]:
# get rid of brackets 
for col in ranks:
    ranks[col] = ranks[col].astype(str).str.replace("[\]\{\}\'\(\)\ \[]","")
ranks.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,player_puuid,rank
0,NFuiAvF0C9YMCBBfgJpDYAaaZqS8IFhnKX5_02QBeZIREl...,Gold
1,ATaIws30cj0UfWN179fxT95zfYUrAAWNPhJHRh6h8_IIYR...,Gold
2,-q5TqZRmRLTIBcRfX9EPDjksLEReE7qtm7i4rplVd6BoGk...,Gold
3,2skPefI2lAflL18iJ3KCyb5fZ7OmIKI-d7iaI0Tc1e3Sjn...,Gold
4,jue-VFnXs4CIrR0kAfkU3dd8voN7im5Z7nJprRX-ELuJJH...,Gold


In [47]:
# check for duplicates
ranks.describe()
ranks = ranks.drop_duplicates()

## Full dataset

In [48]:
# merge matches_melted and ranks
df_full = avg_matches.merge(ranks, on= 'player_puuid', how='left')
df_full

,match_id,timestamp,player_number,player_puuid,rank
0,5857231455,2022-05-05 13:18:51+00:00,player1,ATaIws30cj0UfWN179fxT95zfYUrAAWNPhJHRh6h8_IIYR...,Gold
1,5857099343,2022-05-05 12:47:24+00:00,player1,qxzKB-XDos1g8BVytNYaxzC5QvaXnJ2FHPkMtv0rWG_wmv...,Gold
2,5856620450,2022-05-04 23:45:01+00:00,player1,YEFUr568ruap9KJSRAeT29Fkb34anUma7sX9OeYRxs0-Pi...,Bronze
3,5856716446,2022-05-04 23:09:33+00:00,player1,NFuiAvF0C9YMCBBfgJpDYAaaZqS8IFhnKX5_02QBeZIREl...,Gold
4,5855943984,2022-05-04 18:04:03+00:00,player1,J87wGe7JHWF07jC521xbEiFuRnHFqC5NPbwM_IUEjOgfzj...,Silver
...,...,...,...,...,...
338154,5849195925,2022-04-30 14:08:21+00:00,player10,izgmY6eI5K23lBVVKFBY1F6FAsTnTLeFitB8nVctdtSgzP...,NaN
338155,5847764157,2022-04-29 18:41:38+00:00,player10,4WFU18gcKnX1ZgYxtrLgxNGDwECbazQhoTFTSNtRIScc8p...,NaN
338156,5846127974,2022-04-28 19:52:38+00:00,player10,4roZgIWXMIVWKcqnQtzfKL7kYDcTCBOccV9sCi8ZJ3gpfE...,NaN
338157,5846141732,2022-04-28 19:17:30+00:00,player10,4roZgIWXMIVWKcqnQtzfKL7kYDcTCBOccV9sCi8ZJ3gpfE...,NaN


In [49]:
len(df_full)

338159

In [50]:
len(avg_matches)

338101

## Fully mined AVG players with rank

In [51]:
df=df_full[df_full['rank'].notnull()]
df.head()

,match_id,timestamp,player_number,player_puuid,rank
0,5857231455,2022-05-05 13:18:51+00:00,player1,ATaIws30cj0UfWN179fxT95zfYUrAAWNPhJHRh6h8_IIYR...,Gold
1,5857099343,2022-05-05 12:47:24+00:00,player1,qxzKB-XDos1g8BVytNYaxzC5QvaXnJ2FHPkMtv0rWG_wmv...,Gold
2,5856620450,2022-05-04 23:45:01+00:00,player1,YEFUr568ruap9KJSRAeT29Fkb34anUma7sX9OeYRxs0-Pi...,Bronze
3,5856716446,2022-05-04 23:09:33+00:00,player1,NFuiAvF0C9YMCBBfgJpDYAaaZqS8IFhnKX5_02QBeZIREl...,Gold
4,5855943984,2022-05-04 18:04:03+00:00,player1,J87wGe7JHWF07jC521xbEiFuRnHFqC5NPbwM_IUEjOgfzj...,Silver


In [52]:
len(df)

43133

In [53]:
df.to_csv('df_melted_fullymined_avg.csv', sep='|')

Questions to answer in Network Analysis:

- Why are they popular?
- Why has this node high degree centrality?
- Why this node has high proximity prestige?